# SHRED for ROMs: interpolation - contour plotting

This notebook serves as plotting routine for the Full Order reconstruction, using *pyvista*.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, rcParams
import pickle
from pyforce.tools.functions_list import FunctionsList

import vtk
import pyvista as pv
import dolfinx

pv.start_xvfb()

plt.rcParams.update({
  "text.usetex": True,
  "font.family": "serif"
})

rcParams['text.latex.preamble'] = r'\usepackage{amssymb} \usepackage{amsmath} \usepackage{amsthm} \usepackage{mathtools}'

var_names, is_vector, times = pickle.load(open('msfr_uloff.set', 'rb'))

path_svd = './OfflineSVD/'
u_total = pickle.load(open(path_svd+'u_total.svd', 'rb'))
v_total = pickle.load(open(path_svd+'v_total.svd', 'rb'))
s_total = pickle.load(open(path_svd+'s_total.svd', 'rb'))

var_names = pickle.load(open(path_svd+"var.names", 'rb'))
rescaling_snaps = pickle.load(open(path_svd+"rescaling.svd", 'rb'))

incore_measure = False

if incore_measure:
    path_test = './Test_results_Forecaster/InCore/'
    measurements = pickle.load(open(path_svd+'measurements_incore.data', 'rb'))
else:
    path_test = './Test_results_Forecaster/'
    measurements = pickle.load(open(path_svd+'measurements.data', 'rb'))
    
# Load the results
test_recons, test_ground_truth, _ = pickle.load(open(path_test+'test.results', 'rb'))

if incore_measure:
    path_shred = './SHRED_Forecaster/InCore/'
else:
    path_shred = './SHRED_Forecaster/'
test_datasets = pickle.load(open(path_shred+'datasets.test', 'rb'))

new_t = np.linspace(min(times), max(times), len(measurements['noisy_output'][0,:,0]))

Let us define the tex variables

In [2]:
energy_groups = 6
tex_var_names = [r'\phi_'+str(g+1) for g in range(energy_groups)]

prec_groups = 8
tex_var_names.extend([r'c_'+str(g+1) for g in range(prec_groups)])

dec_groups = 3
tex_var_names.extend([r'd_'+str(g+1) for g in range(dec_groups)])

tex_var_names.extend(['T', 'p', r'\mathbf{u}'])

assert len(tex_var_names) == len(var_names)

Let us a configuration measurements for tikz purposes

In [3]:
# for kk in range(3):
#     fig, ax = plt.subplots(figsize=(7,4))

#     plt.plot(new_t, measurements['noisy_output'][kk, :, 0], 'b')

#     # Remove background
#     fig.patch.set_visible(False)
#     ax.patch.set_visible(False)
#     # ax.axis('off')
    
#     ax.set_xlabel('Time $t$ (s)', fontsize=25)
#     ax.set_ylabel('Measure $y^\Phi_'+str(kk+1)+'$ (s)', fontsize=25)
#     ax.set_xticks([])
#     ax.set_yticks([])

#     fig.savefig(path_test+'measure_'+str(kk)+'.png', format='png', dpi=800, bbox_inches='tight')
#     plt.close(fig)

Let us import the mesh and let us define the functional spaces

In [4]:
from mesh import *
from dolfinx.fem import Function, FunctionSpace, locate_dofs_topological
import ufl
from IPython.display import clear_output

domain, ct, ft = evol_mesh()

fun_spaces = [FunctionSpace(domain, ('Lagrange', 1))]*(len(var_names)-1)
fun_spaces.append(FunctionSpace(domain, ufl.VectorElement("CG", domain.ufl_cell(), 1)))

measured_field = 0

clear_output()

reflector_mark = 20
refl_tags = locate_dofs_topological(fun_spaces[measured_field], ct.dim, ct.find(reflector_mark))
Nh = fun_spaces[measured_field].tabulate_dof_coordinates().shape[0]

Let us define the function to map from *dolfinx.fem.dolfinx* to grid for *pyvista*

In [5]:
def vector_grids(fun: dolfinx.fem.Function, mag_plot: bool, varname='u'):
    
    topology, cells, geometry = dolfinx.plot.create_vtk_mesh(fun.function_space)
    grid = pv.UnstructuredGrid(topology, cells, geometry)

    values = np.zeros((geometry.shape[0], 3))
    values[:, :len(fun)] = np.real(fun.x.array.reshape(geometry.shape[0], len(fun)))
    grid[varname] = values

    if mag_plot:
        warped = grid.warp_by_vector(varname, factor=0.0) 
    else:
        warped = grid.glyph(varname, factor=0.075, tolerance=0.015)
        
    return warped, values

def grids(fun: dolfinx.fem.Function):
    topology, cell_types, geometry = dolfinx.plot.create_vtk_mesh(fun.function_space)
    u_grid = pv.UnstructuredGrid(topology, cell_types, geometry)
    u_grid.point_data['fun'] = fun.x.array[:].real
    u_grid.set_active_scalars('fun')
    return u_grid

Now let us make some plot of the full order fields with the sensors

In [6]:
# from pyforce.tools.write_read import ImportH5

# field_i = measured_field
# snaps_to_plot = ImportH5(fun_spaces[field_i], './dolfinx_data/'+var_names[field_i], var_names[field_i], verbose=False)[0]

# for mu_to_plot in np.arange(99, len(snaps_to_plot), 100):
#     cmap = cm.Spectral_r

#     resolution = [1200, 1200]
#     plotter = pv.Plotter(shape=(1, 1), off_screen=False, border=False, window_size=resolution)
        
#     lab_fontsize = 35
#     title_fontsize = 60
#     zoom = 1.025

#     dict_cb = dict(title = ' ', width = 0.5,
#                     title_font_size=title_fontsize,
#                     label_font_size=lab_fontsize,
#                     color='k',
#                     position_x=0.25, position_y=0.89,
#                     shadow=False) 
#     dict_cb['n_labels'] = 0

#     clim_scale = .01

#     plotter.subplot(0,0)
#     clim = [0,  max(snaps_to_plot(99)) * (1+clim_scale)]
#     dict_cb['title'] = r'FOM - $'+tex_var_names[field_i]+r'(\mathbf{x}; t)$'
#     plotter.add_mesh(grids(snaps_to_plot.map(mu_to_plot)), cmap = cmap, clim = clim, show_edges=False, scalar_bar_args=dict_cb)

#     plotter.add_point_labels(   measurements['mesh'][measurements['location'][:,0]], 
#                                 np.arange(1,4,1), italic=False,
#                                 font_size=40,
#                                 point_color='black',
#                                 shape_color='white',
#                                 shape_opacity=0.85,
#                                 point_size=25,
#                                 render_points_as_spheres=True,
#                                 always_visible=True,
#                                 shadow=False)

#     plotter.view_xy()
#     plotter.camera.zoom(zoom)

#     plotter.screenshot('FOM_'+var_names[field_i]+'_'+str(mu_to_plot)+'.png', transparent_background = True,  window_size=resolution)
#     plotter.close()

Let us plot now the contours of the full test transient for a specific field.

In [7]:
def Plot_SHRED_reconstruction( fom: FunctionsList, shred: FunctionsList,
                         tex_var: str, 
                         time_to_plot: int, title : str,
                         filename: str, clims : list = None,
                         colormaps = None, colormaps_residual = None):  
    
    nrows = 1
    ncols = 3
    
    if colormaps is None:
        _colormaps = cm.jet
        _colormaps_residual = cm.plasma_r
    else:
        _colormaps = colormaps
        _colormaps_residual = colormaps_residual
    
    resolution = [1000 * ncols, 1000 * nrows]
    plotter = pv.Plotter(shape=(nrows, ncols), off_screen=False, border=False, window_size=resolution)
    
    lab_fontsize = 30
    title_fontsize = 40
    zoom = 1.0
    
    dict_cb = dict( width = 0.65, height = 0.15,
                    title_font_size=title_fontsize,
                    label_font_size=lab_fontsize,
                    n_labels=2,
                    color = 'k',
                    position_x=0.175, position_y=0.87,
                    shadow=False) 
    
    # FOM
    plotter.subplot(0,0)
    
    if fom.fun_space.num_sub_spaces > 0:
        is_vector = True
        warped_, _ = vector_grids(fom.map(time_to_plot), mag_plot=True, varname=tex_var)
    else:
        is_vector = False
        warped_ = grids(fom.map(time_to_plot))
    
    if clims is None:
        _clim = [min(fom(time_to_plot)), max(fom(time_to_plot))]
    else:
        _clim = clims
    
    dict_cb['title'] = 'FOM - $'+tex_var+'$'
    plotter.add_mesh(warped_, clim=_clim, cmap=_colormaps, show_edges=False, scalar_bar_args=dict_cb)
    
    plotter.view_xy()
    plotter.camera.zoom(zoom)

    plotter.add_text(title, color= 'k', position=[50, 50], font_size=35)
    
    # SHRED
    plotter.subplot(0,1)
    
    if shred.fun_space.num_sub_spaces > 0:
        assert is_vector is True
        warped_, _ = vector_grids(shred.map(time_to_plot), mag_plot=True, varname=tex_var)
    else:
        assert is_vector is False
        warped_ = grids(shred.map(time_to_plot))
    
    dict_cb['title'] = 'SHRED - $'+tex_var+'$'
    plotter.add_mesh(warped_, clim=_clim, cmap=_colormaps, show_edges=False, scalar_bar_args=dict_cb)
    
    plotter.view_xy()
    plotter.camera.zoom(zoom)

    # Residual
    plotter.subplot(0,2)
    residual = Function(fom.fun_space)
    residual.x.array[:] = np.abs(fom(time_to_plot) - shred(time_to_plot))
    
    if is_vector:
        warped_, _ = vector_grids(residual, mag_plot=True, varname=tex_var)
    else:
        warped_ = grids(residual)
    
    res_clim = [0, max(residual.x.array)]
    dict_cb['title'] = 'Residual - $'+tex_var+'$'
    plotter.add_mesh(warped_, clim=res_clim, cmap=_colormaps_residual, show_edges=False, scalar_bar_args=dict_cb)
    
    plotter.view_xy()
    plotter.camera.zoom(zoom)

    
    plotter.set_background('white', top='white')
        
    ## Save figure
    plotter.screenshot(filename+'.png', transparent_background = True,  window_size=resolution)
    plotter.close()

Let us plot an entire transient to make a video

In [15]:
import os
from tqdm import tqdm

field_i = 19
path_fig = path_test+'Fig/'+var_names[field_i]+'/'
if not os.path.exists(path_fig):
    os.makedirs(path_fig)

kk = 0 # configuration idx
Nmodes = Nmodes = s_total[var_names[0]].shape[0]

# Decoding
v_truth = test_ground_truth[kk][:, 3 + field_i*Nmodes : 3 + (field_i+1)*Nmodes]
v_shred = test_recons[kk][:,3 + field_i*Nmodes : 3 + (field_i+1)*Nmodes]

u_ = u_total[var_names[field_i]]
s_ = np.diag(s_total[var_names[field_i]])

fom = (u_ @ s_ @ v_truth.T) * rescaling_snaps[field_i]
fom_list = FunctionsList(fun_spaces[field_i])
for mu in range(fom.shape[1]):
    fom_list.append(fom[:,mu])

prediction = (u_ @ s_ @ v_shred.T) * rescaling_snaps[field_i]
pred_list = FunctionsList(fun_spaces[field_i])
for mu in range(prediction.shape[1]):
    pred_list.append(prediction[:,mu])
    

# Make plot
for time_to_plot in tqdm(np.arange(0, len(fom_list), 5, dtype=int)):
# for time_to_plot in [0, 50, 100, 250, 500, 700]:
    time = new_t[test_datasets[kk]['idx']][time_to_plot]
    title = 'Time $t={:.2f}$ seconds'.format(time)
    
    # Velocity - U 
    if time > 5:
        clims = [0,0.5]
    else:
        clims = [0, 1.5]
    
    # Temperature - T
    # clims = [860, 1350]
    
    # # Flux 5
    # clims = [1e12, 6e18]
    
    Plot_SHRED_reconstruction(  fom_list, pred_list, tex_var_names[field_i], time_to_plot=time_to_plot, title=title,
                                filename=path_fig+var_names[field_i]+'_time_{:.2f}'.format(time), 
                                # clims = None,
                                clims = clims
                                )

Here a similar way for plotting is reported using matplotlib and its scatter functions.

In [9]:
# fields_to_plot = [0, 6, 9, 17]
# cmaps = [cm.viridis, cm.PiYG, cm.jet, cm.Spectral]

# path='./Figs/'
# if not os.path.exists(path):
#     os.makedirs(path)

# mesh = measurements['mesh']

# for j in [-10]:
# # for j in tqdm(np.arange(0, len(test_indices)+1, 25, dtype=int)):
    
#     fig, axs = plt.subplots(nrows = len(fields_to_plot), ncols = 3, figsize=(6 * 3, 5 * len(fields_to_plot)))

#     axs = axs.reshape(len(fields_to_plot), 3)

#     for kk, field_i in enumerate(fields_to_plot):
#         upca = u_total[var_names[field_i]]
#         spca = s_total[var_names[field_i]]
#         vpca1 = test_ground_truth[:,(field_i*Nmodes)+3:(field_i+1)*Nmodes+3]
#         vpca2 = test_recons[:,(field_i*Nmodes)+3:(field_i+1)*Nmodes+3]

#         u1svd = upca @ np.diag(spca) @ vpca1.T * rescaling_snaps[field_i]
#         u2svd = upca @ np.diag(spca) @ vpca2.T * rescaling_snaps[field_i]

#         # cbar_min = min(u1svd.flatten())
#         # cbar_max = max(u1svd.flatten())
#         cbar_min = min(u1svd[:,j]) #* 1.05
#         cbar_max = max(u1svd[:,j]) #* 1.05

#         scatter_ = axs[kk, 0].scatter(mesh[:,0], mesh[:,1], c=u1svd[:,j], cmap=cmaps[kk], vmin=cbar_min, vmax=cbar_max)
#         plt.colorbar(scatter_)
#         axs[kk, 0].set_xlim(min(mesh[:,0]), max(mesh[:,0]))
#         axs[kk, 0].set_ylim(min(mesh[:,1]), max(mesh[:,1]))
#         axs[kk, 0].set_title(r'Ground Truth - $'+tex_var_names[field_i]+'$' + r' at $t = {:.2f}$ s'.format(new_t[test_indices[j]]), fontsize=15)

#         scatter_ = axs[kk, 1].scatter(mesh[:,0], mesh[:,1], c=u2svd[:,j], cmap=cmaps[kk], vmin=cbar_min, vmax=cbar_max)
#         plt.colorbar(scatter_)
#         axs[kk, 1].set_xlim(min(mesh[:,0]), max(mesh[:,0]))
#         axs[kk, 1].set_ylim(min(mesh[:,1]), max(mesh[:,1]))
#         axs[kk, 1].set_title(r'SHRED - $'+tex_var_names[field_i]+'$', fontsize=15)

#         scatter_ = axs[kk, 2].scatter(mesh[:,0], mesh[:,1], c=np.abs(u1svd[:,j] - u2svd[:,j]) , cmap=cm.hot)
#         plt.colorbar(scatter_)
#         axs[kk, 2].set_xlim(min(mesh[:,0]), max(mesh[:,0]))
#         axs[kk, 2].set_ylim(min(mesh[:,1]), max(mesh[:,1]))
#         axs[kk, 2].set_title(r'Residual - $'+tex_var_names[field_i]+'$', fontsize=15)

#     for ax in axs.flatten():
#         ax.set_xticks([], [])
#         ax.set_yticks([], [])

#     plt.tight_layout()
#     # fig.savefig(path+'Contours_{:.2f}'.format(new_t[test_indices[j]])+'.png', format='png', dpi=150, bbox_inches='tight')
#     # plt.close(fig)